In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
# Load the dataset
df = pd.read_csv('uber.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         200000 non-null  int64  
 1   key                200000 non-null  object 
 2   fare_amount        200000 non-null  float64
 3   pickup_datetime    200000 non-null  object 
 4   pickup_longitude   200000 non-null  float64
 5   pickup_latitude    200000 non-null  float64
 6   dropoff_longitude  199999 non-null  float64
 7   dropoff_latitude   199999 non-null  float64
 8   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 13.7+ MB


In [4]:
#pickup_datetime is not in required data format
df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])

In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   Unnamed: 0         200000 non-null  int64              
 1   key                200000 non-null  object             
 2   fare_amount        200000 non-null  float64            
 3   pickup_datetime    200000 non-null  datetime64[ns, UTC]
 4   pickup_longitude   200000 non-null  float64            
 5   pickup_latitude    200000 non-null  float64            
 6   dropoff_longitude  199999 non-null  float64            
 7   dropoff_latitude   199999 non-null  float64            
 8   passenger_count    200000 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(2), object(1)
memory usage: 13.7+ MB


In [6]:
df.describe()

Unnamed: 0    fare_amount  pickup_longitude  pickup_latitude  \
count  2.000000e+05  200000.000000     200000.000000    200000.000000   
mean   2.771250e+07      11.359955        -72.527638        39.935885   
std    1.601382e+07       9.901776         11.437787         7.720539   
min    1.000000e+00     -52.000000      -1340.648410       -74.015515   
25%    1.382535e+07       6.000000        -73.992065        40.734796   
50%    2.774550e+07       8.500000        -73.981823        40.752592   
75%    4.155530e+07      12.500000        -73.967154        40.767158   
max    5.542357e+07     499.000000         57.418457      1644.421482   

       dropoff_longitude  dropoff_latitude  passenger_count  
count      199999.000000     199999.000000    200000.000000  
mean          -72.525292         39.923890         1.684535  
std            13.117408          6.794829         1.385997  
min         -3356.666300       -881.985513         0.000000  
25%           -73.991407         40.733823         1.000000  
50%           -73.980093         40.753042         1.000000  
75%           -73.963658         40.768001         2.000000  
max          1153.572603        872.697628       208.000000

In [7]:
df.head()

Unnamed: 0                            key  fare_amount  \
0    24238194    2015-05-07 19:52:06.0000003          7.5   
1    27835199    2009-07-17 20:04:56.0000002          7.7   
2    44984355   2009-08-24 21:45:00.00000061         12.9   
3    25894730    2009-06-26 08:22:21.0000001          5.3   
4    17610152  2014-08-28 17:47:00.000000188         16.0   

            pickup_datetime  pickup_longitude  pickup_latitude  \
0 2015-05-07 19:52:06+00:00        -73.999817        40.738354   
1 2009-07-17 20:04:56+00:00        -73.994355        40.728225   
2 2009-08-24 21:45:00+00:00        -74.005043        40.740770   
3 2009-06-26 08:22:21+00:00        -73.976124        40.790844   
4 2014-08-28 17:47:00+00:00        -73.925023        40.744085   

   dropoff_longitude  dropoff_latitude  passenger_count  
0         -73.999512         40.723217                1  
1         -73.994710         40.750325                1  
2         -73.962565         40.772647                1  
3         -73.965316         40.803349                3  
4         -73.973082         40.761247                5

In [8]:
# Preprocess the dataset
# Remove any rows with missing values
df.dropna(inplace=True)

In [9]:
# Identify and remove outliers
# For example, we can use the following code to remove any rides with a fare amount greater than $100
df = df[df['fare_amount'] < 100]

In [10]:
# Check the correlation between the features
# For example, we can use the following code to generate a correlation matrix
numeric_df = df.select_dtypes(include=['number'])

# Check the correlation between the features
corr_matrix = numeric_df.corr()
print(corr_matrix)

                   Unnamed: 0  fare_amount  pickup_longitude  pickup_latitude  \
Unnamed: 0           1.000000     0.000045          0.000289        -0.000387   
fare_amount          0.000045     1.000000          0.007301        -0.005944   
pickup_longitude     0.000289     0.007301          1.000000        -0.816269   
pickup_latitude     -0.000387    -0.005944         -0.816269         1.000000   
dropoff_longitude    0.000322     0.006246          0.832830        -0.774562   
dropoff_latitude     0.000219    -0.008269         -0.846126         0.702054   
passenger_count      0.002252     0.012056         -0.000432        -0.001546   

                   dropoff_longitude  dropoff_latitude  passenger_count  
Unnamed: 0                  0.000322          0.000219         0.002252  
fare_amount                 0.006246         -0.008269         0.012056  
pickup_longitude            0.832830         -0.846126        -0.000432  
pickup_latitude            -0.774562          0.702054 

In [11]:
# Select the most important features
# For example, we can select the features that have a correlation of greater than 0.5 with the target variable
features = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count']

In [12]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df[features], df['fare_amount'], test_size=0.25, random_state=42)

In [13]:
# Train the linear regression model
linear_regression_model = LinearRegression()
linear_regression_model.fit(X_train, y_train)

LinearRegression()

In [14]:
# Train the random forest regression model
random_forest_regression_model = RandomForestRegressor()
random_forest_regression_model.fit(X_train, y_train)

RandomForestRegressor()

In [15]:
# Make predictions on the test set
linear_regression_predictions = linear_regression_model.predict(X_test)
random_forest_regression_predictions = random_forest_regression_model.predict(X_test)

In [16]:
# Evaluate the models
linear_regression_r2 = r2_score(y_test, linear_regression_predictions)
linear_regression_rmse = mean_squared_error(y_test, linear_regression_predictions)**0.5

In [17]:
random_forest_regression_r2 = r2_score(y_test, random_forest_regression_predictions)
random_forest_regression_rmse = mean_squared_error(y_test, random_forest_regression_predictions)**0.5

In [18]:
# Print the results
print('Linear regression:')
print('R2:', linear_regression_r2)
print('RMSE:', linear_regression_rmse)

Linear regression:
R2: -6.819297748106301e-05
RMSE: 9.343839813756933


In [19]:
print('Random forest regression:')
print('R2:', random_forest_regression_r2)
print('RMSE:', random_forest_regression_rmse)

Random forest regression:
R2: 0.807106781827321
RMSE: 4.103638129632614
